# **Explicación de las Variables Agregadas**
Se incluyeron variables que mejoran la predicción de tendencias en el mercado:

**Momentum**: Evalúa la tasa de cambio de precios, útil para detectar tendencias alcistas o bajistas.

**MACD Signal**: Indicador clave en análisis técnico que ayuda a identificar la dirección y la fuerza de una tendencia.

**Risk Factor**: Medida de volatilidad basada en el True Range, esencial para evaluar el riesgo del activo.

**Stochastic D**: Proporciona información sobre el momentum del precio en relación con su rango en un período determinado.

**Relative Strength Index (RSI)**: Indica si un activo está sobrecomprado o sobrevendido.

**Log Price**: Permite normalizar los precios para evitar problemas de escalabilidad.

**Volume Trend (OBV)**: Relaciona el volumen con los cambios de precio, útil para confirmar tendencias.

**Average True Range (ATR)**: Determina la volatilidad general del mercado.

In [1]:
###DEBUG###

! pip install git+https://github.com/quantiacs/toolbox.git 2>/dev/null

# decrease height
from IPython.display import Javascript
display(Javascript('google.colab.output.setIframeHeight(0, true, {maxHeight: 100})'))

  Cloning https://github.com/quantiacs/toolbox.git to /tmp/pip-req-build-4j63g9uu
  Resolved https://github.com/quantiacs/toolbox.git to commit a1e6351446cd936532af185fb519ef92f5b1ac6d
  Preparing metadata (setup.py) ... done


<IPython.core.display.Javascript object>

In [ ]:
###DEBUG###
!wget http://prdownloads.sourceforge.net/ta-lib/ta-lib-0.4.0-src.tar.gz
!tar -xzvf ta-lib-0.4.0-src.tar.gz
%cd ta-lib
!./configure --prefix=/usr
! make
!make install
!pip install Ta-Lib

# test import
import talib

# decrease height
from IPython.display import Javascript
display(Javascript('google.colab.output.setIframeHeight(0, true, {maxHeight: 100})'))

In [3]:
###DEBUG###
import os

os.environ['API_KEY'] = "f0b0a280-9f57-4c40-9dbd-9df958f2bbdc"
os.environ['NONINTERACT'] = 'True'

In [ ]:
import xgboost as xgb
import qnt.data as qndata
import qnt.ta as qnta
import xarray as xr
import numpy as np

# =============================
# Cargar Datos del Mercado
# =============================
stock_data = qndata.stocks.load_spx_data(min_date="2006-01-01")

# Filtrar solo los activos líquidos
is_liquid = stock_data.sel(field='is_liquid')
stock_data = stock_data.where(is_liquid, drop=True)

# =============================
# Construcción de Indicadores
# =============================
def construir_indicadores(datos_mercado):
    """Construcción de indicadores para la predicción del mercado."""

    # Momento del Mercado
    momento = qnta.roc(qnta.lwma(datos_mercado.sel(field="close"), 60), 1)
    momento = momento.expand_dims(field=["momento"])

    # Señal MACD
    macd_principal, macd_senal, macd_hist = qnta.macd(datos_mercado, 12, 26, 9)
    macd_senal = macd_senal.sel(field="close").expand_dims(field=["macd_senal"])

    # Factor de Riesgo
    factor_riesgo = qnta.tr(datos_mercado.sel(field="high"), datos_mercado.sel(field="low"), datos_mercado.sel(field="close"))
    factor_riesgo = factor_riesgo / datos_mercado.sel(field="close")
    factor_riesgo = qnta.lwma(factor_riesgo, 14).expand_dims(field=["factor_riesgo"])

    # Estocástico D
    estocastico_k, estocastico_d = qnta.stochastic(datos_mercado.sel(field="high"), datos_mercado.sel(field="low"), datos_mercado.sel(field="close"), 14)
    estocastico_d = estocastico_d.expand_dims(field=["estocastico_d"])

    # Índice de Fuerza Relativa (RSI)
    fuerza_relativa = qnta.rsi(datos_mercado.sel(field="close")).expand_dims(field=["rsi"])

    # Precio Logarítmico
    precio_log = datos_mercado.sel(field="close").ffill("time").bfill("time").fillna(0)
    precio_log = np.log(precio_log).expand_dims(field=["precio_log"])

    # Tendencia de Volumen (OBV)
    tendencia_volumen = qnta.obv(datos_mercado.sel(field="close"), datos_mercado.sel(field="vol")).expand_dims(field=["tendencia_volumen"])

    # Rango Promedio Verdadero (ATR)
    rango_promedio_verdadero = qnta.atr(datos_mercado.sel(field="high"), datos_mercado.sel(field="low"), datos_mercado.sel(field="close"), 14).expand_dims(field=["atr"])

    # Agrupar Indicadores
    conjunto_caracteristicas = xr.concat([momento, macd_senal, factor_riesgo, estocastico_d, fuerza_relativa, precio_log, tendencia_volumen, rango_promedio_verdadero], dim="field")
    return conjunto_caracteristicas.transpose("time", "field", "asset")

# **Técnica de Machine Learning**

Elegí XGBoost porque es un algoritmo de aprendizaje automático basado en árboles de decisión que ofrece un alto rendimiento y precisión en la clasificación de datos financieros. XGBoost maneja bien datos con características correlacionadas y es robusto frente al sobreajuste gracias a su regularización integrada. Además, es más eficiente en términos computacionales y se adapta mejor a conjuntos de datos grandes y ruidosos, lo que es clave en la predicción de mercados. Su capacidad de modelar relaciones no lineales y capturar patrones complejos lo hace superior a modelos más simples

In [3]:
# =============================
# Creación del Modelo de ML
# =============================
def construir_modelo():
    """Modelo mejorado: XGBoost en lugar de MLPClassifier."""
    modelo = xgb.XGBClassifier(
        n_estimators=100,
        learning_rate=0.1,
        max_depth=5,
        subsample=0.8,
        colsample_bytree=0.8,
        random_state=42
    )
    return modelo

# **Se utiliza S&P 500 y los activos se filtraron solo a los activos líquidos**

He decidido cargar todos los activos del S&P 500 para evitar problemas con activos no disponibles y usar todo el mercado disponible.




In [ ]:
# =============================
# Aplicar Construcción de Indicadores y Definir Clases
# =============================
indicadores = construir_indicadores(stock_data)
target_classes = get_target_classes(stock_data)

Se aceptan cortos

In [6]:
# =============================
# Definir Clases Objetivo
# =============================
def get_target_classes(data):
    """Define las clases objetivo para predicción de precios en largos y cortos."""

    price_current = data.sel(field="close")
    price_future  = qnta.shift(price_current, -1)

    class_positive = 1  # Sube más de 0.1%
    class_neutral = 0   # Fluctúa dentro de ±0.1%
    class_negative = -1 # Baja más de 0.1%

    target_price_up = xr.where(
        price_future > price_current * 1.001, class_positive,
        xr.where(price_future < price_current * 0.999, class_negative, class_neutral)
    )

    return target_price_up